#### Table of Contents <a id='toc'></a>

### [Introduction](#intro)
### [Business & Data Understanding](#bdu)
### [Prepare Data](#prepare_data)
### [Data Modelling](#data_modelling)
### [Evaluation](#evaluation)

### Introduction <a id='intro'></a>
[back](#toc)


https://www.kaggle.com/c/pubg-finish-placement-prediction/overview

### Business & Data Understanding <a id='bdu'></a>
[back](#toc)

### Prepare Data <a id='prepare_data'></a>
[back](#toc)

### Data Modelling <a id='data_modelling'></a>
[back](#toc)

### Evaluation <a id='evaluation'></a>
[back](#toc)